In [1]:
!pip install gensim

  Using cached gensim-4.2.0-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached numpy-1.23.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (17.1 MB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)
  Using cached scipy-1.9.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (43.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pythonwhat 2.23.1 requires dill~=0.2.7.1, but you have dill 0.3.3 which is incompatible.
pythonwhat 2.23.1 requires jinja2~=2.10, but you have jinja2 3.0.1 which is incompatible.
--- Logging error ---
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/pip/_internal/utils/logging.py", line 177, in emit
    self.console.print(renderable, overflow="ignore", crop=False, style=style)
  File "/usr/local/lib/python3.8/dist-packages/pip/_vendor/rich/console

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('tweets_translated.csv')

In [4]:
df.dropna(subset=['tweet'],inplace=True)

In [5]:
df['tweet'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', x))

In [153]:
df['date'] = pd.to_datetime(df['date'])

In [156]:
df = df.sort_values('date')

In [171]:
dic_week = {}
date = df.date.unique()[4]
week = 1
while date in df.date.unique():
    end_week = date + pd.Timedelta(6, 'd')
    dic_week[f'week: {week}'] = (date, end_week)
    week += 1 
    date = end_week + pd.Timedelta(1, 'd')
dic_week

{'week: 1': (numpy.datetime64('2022-07-18T00:00:00.000000000'),
  Timestamp('2022-07-24 00:00:00')),
 'week: 2': (Timestamp('2022-07-25 00:00:00'),
  Timestamp('2022-07-31 00:00:00')),
 'week: 3': (Timestamp('2022-08-01 00:00:00'),
  Timestamp('2022-08-07 00:00:00'))}

In [189]:
for x, y in dic_week.items():
    t = df[(df.date <= y[1]) & (df.date >= y[0])]
    print(t.date.unique())

['2022-07-18T00:00:00.000000000' '2022-07-19T00:00:00.000000000'
 '2022-07-20T00:00:00.000000000' '2022-07-23T00:00:00.000000000'
 '2022-07-24T00:00:00.000000000']
['2022-07-25T00:00:00.000000000' '2022-07-26T00:00:00.000000000'
 '2022-07-27T00:00:00.000000000' '2022-07-28T00:00:00.000000000'
 '2022-07-29T00:00:00.000000000' '2022-07-30T00:00:00.000000000'
 '2022-07-31T00:00:00.000000000']
['2022-08-01T00:00:00.000000000' '2022-08-02T00:00:00.000000000'
 '2022-08-03T00:00:00.000000000' '2022-08-04T00:00:00.000000000'
 '2022-08-05T00:00:00.000000000']


In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/repl/nltk_data...


True

In [7]:
stemmer = SnowballStemmer('english')

In [203]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    
    for token in gensim.utils.simple_preprocess(text):
        #We accept the word max since it is used for driver Max Verstappen
        if (token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token != 'live') or token == 'max':
            
            result.append(lemmatize_stemming(token))
            
    return result

In [ ]:
from gensim import corpora, models
from tqdm import tqdm
week_topics = {}

for week, days in tqdm(dic_week.items()):
    topics_per_week = {}
    week_df = df[(df.date <= days[1]) & (df.date >= days[0])]
    processed = week_df['tweet'].map(preprocess)
    dictionary = gensim.corpora.Dictionary(processed)
    
    # filter words that abear in less than 20% off tweets and keep all popular words
    dictionary.filter_extremes(no_below=0.2, no_above=0.45, keep_n=100000)
    
    # Create a Bag-of-Words model fot every tweet
    bow_corpus = [dictionary.doc2bow(tweet) for tweet in processed]
    
    # creating a TF_IDF model
    tfidf = models.TfidfModel(bow_corpus)
    
    # Apply transformation to the entire corpus call
    corpus_tfidf = tfidf[bow_corpus]
    
    # LDA modeling 
    lda_model = gensim.models.LdaMulticore(corpus_tfidf,
                                         num_topics=10,
                                         id2word=dictionary,
                                         alpha=1,
                                         passes=2)
    
    
    for idx, topic in lda_model.print_topics(-1):
        topics_per_week[idx] = topic
        
    week_topics[week] = topics_per_week

In [202]:
week_topics

{'week: 1': {0: '0.012*"grand" + 0.012*"live" + 0.012*"prix" + 0.012*"stream" + 0.012*"watch" + 0.011*"frenchgp" + 0.010*"𝐋𝐈𝐕𝐄" + 0.009*"mercedesamgf" + 0.008*"french" + 0.008*"franc"',
  1: '0.022*"live" + 0.015*"𝐋𝐈𝐕𝐄" + 0.012*"stream" + 0.012*"prix" + 0.012*"watch" + 0.011*"frenchgp" + 0.011*"grand" + 0.009*"lenovo" + 0.009*"franc" + 0.009*"french"',
  2: '0.011*"live" + 0.010*"franc" + 0.010*"grand" + 0.009*"watch" + 0.009*"prix" + 0.008*"𝐋𝐈𝐕𝐄" + 0.008*"lenovo" + 0.008*"frenchgp" + 0.007*"stream" + 0.006*"lewishamilton"',
  3: '0.018*"live" + 0.013*"grand" + 0.010*"frenchgp" + 0.010*"stream" + 0.010*"prix" + 0.010*"𝐋𝐈𝐕𝐄" + 0.010*"watch" + 0.008*"mercedesamgf" + 0.007*"franc" + 0.007*"french"',
  4: '0.014*"frenchgp" + 0.014*"live" + 0.011*"stream" + 0.010*"prix" + 0.010*"grand" + 0.010*"𝐋𝐈𝐕𝐄" + 0.009*"watch" + 0.008*"franc" + 0.007*"lenovo" + 0.007*"french"',
  5: '0.014*"frenchgp" + 0.013*"live" + 0.011*"watch" + 0.010*"prix" + 0.010*"grand" + 0.009*"franc" + 0.009*"stream" + 0.009

In [133]:
df_date = df[df.date == '2022-07-28']
processed = df_date['tweet'].map(preprocess)

In [97]:
processed = df['tweet'].map(preprocess)

In [134]:
processed[:10]

157970                       [emilkolega, hamilton, russel]
157971    [scene, indianapoli, world, famous, themelodyi...
157972                 [enthusto, like, skid, mark, underp]
157973                            [benzytwt, master, benzi]
157974    [zero_formula, say, innittwt, sure, major, smi...
157975    [miss, thorough, unafraid, platform, leav, spo...
157976                     [vega, marshal, vitalvega, sign]
157977    [indycaronnbc, cgrteam, alexpalou, zack, brown...
157978    [of_snitch, peterarrindel, gdb_lpn, vega, hell...
157979                                       [race, denial]
Name: tweet, dtype: object

In [135]:
dictionary = gensim.corpora.Dictionary(processed)

In [136]:
# filter words that abear in less than 20% off tweets and keep all popular words
dictionary.filter_extremes(no_below=0.2, no_above=0.5, keep_n=100000)

In [137]:
# Create a Bag-of-Words model fot every tweet
bow_corpus = [dictionary.doc2bow(tweet) for tweet in processed]

In [138]:
# creating a TF_IDF model
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
print(tfidf)

TfidfModel<num_docs=21789, num_nnz=142798>


In [139]:
# Apply transformation to the entire corpus call
corpus_tfidf = tfidf[bow_corpus]
print(corpus_tfidf[0])

[(0, 0.6883612652149576), (1, 0.40181928382705734), (2, 0.6039039921182618)]


In [140]:
from pprint import pprint

for tweet in corpus_tfidf:
    pprint(tweet)
    break

[(0, 0.6883612652149576), (1, 0.40181928382705734), (2, 0.6039039921182618)]


In [196]:
# LDA modeling 
lda_model = gensim.models.LdaMulticore(bow_corpus,
                                     num_topics=10,
                                     id2word=dictionary,
                                     alpha=0.75,
                                     eta=0.5,
                                     passes=2)

  File "/home/repl/.local/lib/python3.8/site-packages/gensim/models/ldamodel.py", line 721, in inference
    phinorm = np.dot(expElogthetad, expElogbetad) + epsilon
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 109, in worker
    initializer(*initargs)
  File "<__array_function__ internals>", line 180, in dot
  File "/home/repl/.local/lib/python3.8/site-packages/gensim/models/ldamulticore.py", line 344, in worker_e_step
    worker_lda.sync_state()
KeyboardInterrupt
  File "/home/repl/.local/lib/python3.8/site-packages/gensim/models/ldamodel.py", line 721, in inference
    phinorm = np.dot(expElogthetad, expElogbetad) + epsilon
  File "/usr/lib/python3.8/multiprocessing/pool.py", line 109, in worker
    initializer(*initargs)
  File "<__array_function__ internals>", line 180, in dot
  File "/home/repl/.local/lib/python3.8/site-packages/gensim/models/ldamulticore.py", line 344, in worker_e_step
    worker_lda.sync_state()
  File "/home/repl/.local/lib/python3.8/site-packages/

In [197]:
for idx, topic in lda_model.print_topics(-1):
    print(f'Topics {idx} \nWords: {topic}')
    print('\n')

Topics 0 
Words: 0.030*"formula" + 0.018*"team" + 0.014*"astonmartinf" + 0.013*"alpinef" + 0.011*"oscarpiastri" + 0.008*"williamsrac" + 0.008*"alex_albon" + 0.007*"alonso" + 0.007*"driver" + 0.006*"race"


Topics 1 
Words: 0.036*"formula" + 0.017*"team" + 0.011*"astonmartinf" + 0.010*"oscarpiastri" + 0.009*"williamsrac" + 0.009*"alpinef" + 0.008*"alpin" + 0.008*"alonso" + 0.007*"alex_albon" + 0.007*"race"


Topics 2 
Words: 0.031*"formula" + 0.017*"team" + 0.010*"astonmartinf" + 0.009*"oscarpiastri" + 0.008*"alpinef" + 0.007*"alonso" + 0.006*"alpin" + 0.006*"max" + 0.006*"driver" + 0.006*"year"


Topics 3 
Words: 0.041*"formula" + 0.012*"team" + 0.010*"astonmartinf" + 0.010*"williamsrac" + 0.009*"race" + 0.008*"alex_albon" + 0.008*"alpin" + 0.007*"max" + 0.007*"alpinef" + 0.007*"alonso"


Topics 4 
Words: 0.031*"formula" + 0.014*"team" + 0.009*"race" + 0.009*"astonmartinf" + 0.009*"max" + 0.008*"alpinef" + 0.008*"driver" + 0.008*"verstappen" + 0.007*"alonso" + 0.006*"ferrari"


Topics 

In [144]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf,
                                     num_topics=10,
                                     id2word=dictionary,
                                     passes=2)

In [145]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print(f'Topics {idx} \nWords: {topic}')
    print('\n')

Topics 0 
Words: 0.011*"thankyouseb" + 0.011*"vettel" + 0.011*"formula" + 0.010*"retir" + 0.009*"sebastian" + 0.008*"driver" + 0.008*"season" + 0.007*"team" + 0.005*"time" + 0.005*"miss"


Topics 1 
Words: 0.023*"scuderiaferrari" + 0.021*"thank" + 0.015*"formula" + 0.009*"vettel" + 0.008*"miss" + 0.008*"great" + 0.007*"time" + 0.006*"thankyouseb" + 0.006*"retir" + 0.006*"driver"


Topics 2 
Words: 0.022*"thank" + 0.019*"miss" + 0.015*"sebastian" + 0.015*"vettel" + 0.014*"formula" + 0.011*"retir" + 0.010*"legend" + 0.007*"announc" + 0.007*"season" + 0.007*"world"


Topics 3 
Words: 0.010*"time" + 0.009*"best" + 0.009*"formula" + 0.008*"vettel" + 0.007*"sebastian" + 0.007*"retir" + 0.006*"driver" + 0.006*"season" + 0.006*"like" + 0.005*"great"


Topics 4 
Words: 0.011*"retir" + 0.011*"vettel" + 0.011*"formula" + 0.009*"announc" + 0.007*"sebastian" + 0.007*"nooooooo" + 0.007*"season" + 0.005*"career" + 0.005*"team" + 0.005*"sebastián"


Topics 5 
Words: 0.011*"formula" + 0.009*"retir" + 0